In [1]:
import cv2
import dlib
import os
import pandas as pd
import numpy as np

# Load the detector and predictor from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Initialize the webcam feed
cap = cv2.VideoCapture(0)

max_frames = 30
num_videos = 8

def display_countdown():
    for count in reversed(range(1, 4)):
        ret, frame = cap.read()
        if not ret:
            break
        cv2.putText(frame, str(count), (int(cap.get(3)) // 2 - 30, int(cap.get(4)) // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        cv2.imshow("Frame", frame)
        cv2.waitKey(1000)

def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def process_frame_landmarks(frame_landmarks):
    distances = []
    for i in range(68):
        for j in range(i + 1, 68):
            distances.append(calculate_distance(frame_landmarks[i], frame_landmarks[j]))
    return distances

dataset_root = 'Dataset/Train/Rohan'

# Create the dataset root directory if it doesn't exist
if not os.path.exists(dataset_root):
    os.makedirs(dataset_root)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

for video_num in range(1, num_videos + 1):
    display_countdown()

    video_dir = os.path.join(dataset_root, f'Video_{video_num}')
    if not os.path.exists(video_dir):
        os.makedirs(video_dir)

    video_filename = f'trial{video_num}.avi'
    video_filepath = os.path.join(video_dir, video_filename)
    out = cv2.VideoWriter(video_filepath, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    frame_count = 0
    all_frame_distances = []

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        for face in faces:
            landmarks = predictor(gray, face)
            frame_landmarks = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]
            frame_distances = process_frame_landmarks(frame_landmarks)
            all_frame_distances.append(frame_distances)

            # Draw each landmark point for visualization
            for n in range(68):
                x, y = frame_landmarks[n]
                cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)

        cv2.imshow("Frame", frame)
        out.write(frame)

        frame_filename = os.path.join(video_dir, f'frame_{frame_count:04d}.png')
        cv2.imwrite(frame_filename, frame)

        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    out.release()

    # Check if all_frame_distances is empty
    if not all_frame_distances:
        print(f"No landmarks detected in video {video_num}. Skipping normalization and CSV creation.")
        continue

    # Normalization
    all_frame_distances = np.array(all_frame_distances)
    min_val = np.min(all_frame_distances)
    max_val = np.max(all_frame_distances)
    normalized_distances = (all_frame_distances - min_val) / (max_val - min_val) if max_val > min_val else all_frame_distances

    # Reshape for CSV
    normalized_distances = normalized_distances.T  # Shape will be (2278, 30)
    normalized_csv_filename = f'normalized_landmarks_video_{video_num}.csv'
    normalized_csv_filepath = os.path.join(video_dir, normalized_csv_filename)
    pd.DataFrame(normalized_distances).to_csv(normalized_csv_filepath, index=False)

cap.release()
cv2.destroyAllWindows()


2023-12-08 15:13:39.366 Python[10515:835107] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Define the function to load data
def load_csv_from_folders(folder_path):
    all_data = []
    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                if file_path.endswith('.csv'):
                    data = pd.read_csv(file_path)
                    all_data.append(data)
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

# Paths to training and testing data folders
training_folder_path = '/Users/ishwarjadhav/Jupyter-Notebook/FGR5/Dataset/Train'
testing_folder_path = '/Users/ishwarjadhav/Jupyter-Notebook/FGR5/Dataset/Test'

# Load your data
X_train = load_csv_from_folders(training_folder_path)
X_test = load_csv_from_folders(testing_folder_path)


y_train = []  #  labels
y_test = []   #  labels

# Initialize 1NN model
model = KNeighborsClassifier(n_neighbors=1)

# Train the model (in 1NN, this is just storing data)
model.fit(X_train, y_train)

# Test the model
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")



NameError: name 'os' is not defined

In [6]:
import pandas as pd
from sklearn.cluster import KMeans
import os

# Function to load CSV files from subfolders
def load_csv_from_folders(folder_path):
    all_data = []
    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file)
                if file_path.endswith('.csv'):
                    data = pd.read_csv(file_path)
                    all_data.append(data)
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

# Load your data
training_folder_path = '/Users/ishwarjadhav/Jupyter-Notebook/FGR5/Dataset/Train/Ishwar'
testing_folder_path = '/Users/ishwarjadhav/Jupyter-Notebook/FGR5/Dataset/Test/Ishwar'

X_train = load_csv_from_folders(training_folder_path)
X_test = load_csv_from_folders(testing_folder_path)


k = 3  # Example, you can choose a different number
n_init_value = 10
kmeans = KMeans(n_clusters=k, n_init=n_init_value)
kmeans.fit(X_train)
# Predict the clusters for the test data
test_clusters = kmeans.predict(X_test)

# View the cluster assignments for the test data
print("Cluster assignments for test data:")
print(test_clusters)

# Optional: View the cluster centers
print("\nCluster Centers:")
print(kmeans.cluster_centers_)

Cluster assignments for test data:
[1 1 2 ... 1 1 1]

Cluster Centers:
[[0.71650131 0.72280316 0.72755779 0.72619988 0.72264307 0.72372754
  0.72275512 0.72327529 0.72221655 0.7220549  0.72463278 0.72417241
  0.72716326 0.72868519 0.73245323 0.73274534 0.73213815 0.7303349
  0.72871478 0.72689135 0.72672719 0.72733242 0.72792691 0.72998376
  0.72947444 0.72983964 0.73080851 0.73170763 0.73142004 0.7315156 ]
 [0.18688492 0.18947663 0.19167905 0.19252777 0.19362673 0.1936587
  0.19295918 0.19325572 0.19284457 0.19169405 0.19286924 0.19223763
  0.19252163 0.19194084 0.19229016 0.19183442 0.19195681 0.19191939
  0.19113104 0.19096605 0.19094553 0.19098543 0.19107232 0.19153561
  0.19145963 0.19169335 0.19170171 0.19237285 0.19217273 0.19239923]
 [0.4564935  0.46097233 0.46461016 0.4639889  0.4600903  0.46102606
  0.46033462 0.46110358 0.45960915 0.46116043 0.4618424  0.46191529
  0.46391756 0.46542299 0.46826694 0.46881072 0.46848932 0.46739138
  0.46633475 0.46484436 0.46496383 0.46578979

In [9]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

# Load the training data
train_data = []
for i in range(1, 9):
    folder_name = 'training_folder_' + str(i)
    for j in range(1, 2279):
        file_name = 'Dataset' + '/Train' + '' + str(j) + '.csv'
        df = pd.read_csv(file_name)
        train_data.append(df)

# Load the testing data
test_data = []
for i in range(1, 3):
    folder_name = 'testing_folder_' + str(i)
    for j in range(1, 2279):
        file_name = folder_name + '/file_' + str(j) + '.csv'
        df = pd.read_csv(file_name)
        test_data.append(df)

# Create the 1NN model
model = KNeighborsClassifier(n_neighbors=1)

# Fit the model using the training data
X_train = pd.concat(train_data)
y_train = [i for i in range(1, 9) for j in range(2278)]
model.fit(X_train, y_train)

# Predict the class labels for the testing data
X_test = pd.concat(test_data)
y_pred = model.predict(X_test)

# Print the predicted class labels
print(y_pred)


NameError: name 'train_data' is not defined

In [12]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os

# Assuming you have similar folders for testing data
train_folder = 'Dataset/Train/Ishwar'
test_folder = 'Dataset/Test/Ishwar'

def load_data(folder):
    all_data = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                if file.endswith('.csv'):
                    data = pd.read_csv(os.path.join(subfolder_path, file))
                    all_data.append(data.values)
    return np.vstack(all_data)

# Load training and testing data
X_train = load_data(train_folder)
y_train = [label1, label2, ...] # You need to provide the labels for training data
X_test = load_data(test_folder)

# Implementing 1NN Classifier
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train, y_train)

# Predicting and Evaluating
predicted_labels = model.predict(X_test)

# If you have ground truth labels for testing data
y_test = [test_label1, test_label2, ...] # Ground truth labels for testing data
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy}")

# If you don't have ground truth labels, you can explore the results qualitatively


NameError: name 'label1' is not defined